In [15]:
from typing import Set
import re
from typing import NamedTuple

In [16]:
def tokenize(text: str) -> Set[str]:
    text = text.lower() # convert to lowercase
    all_words = re.findall("\w+", text) # extract the words
    return set(all_words)

assert tokenize("Data Science is science") == {"data", "science", "is"}

In [17]:
class Message(NamedTuple):
    text: str
    is_spam: bool